# Importando as Bibliotecas

In [ ]:
import pandas as pd
import numpy as np
import string

from scipy import spatial

from sklearn.feature_extraction.text import CountVectorizer

# Lendo os Datasets

In [ ]:
arquivos = ['celulares', 'fogoes', 'geladeiras', 'notebooks', 'tvs']

lista_df_pp = []
for arquivo in arquivos:

    df_pp = pd.read_csv(f"Dados/Produtos Processados/pp_{arquivo}.csv", dtype = {'ean': str})
    lista_df_pp.append(df_pp)

# Aplicando o BoW

In [ ]:
def remove_pontuacao(texto):
    
    texto_sp = texto.translate(str.maketrans('', '', string.punctuation))
    
    return texto_sp   


def formatar_entrada_bow(dados, mf = 1000):
    
    cv = CountVectorizer(
                         lowercase = True,
                         strip_accents = 'unicode',
                         max_features = mf
                        )

    cv.fit(dados)
    dados_transformados = cv.transform(dados).toarray()

    #X = matriz.fit_transform(dados).toarray()
    
    return cv, dados_transformados


def calcular_dis_cos(vetor_1, vetor_2):
                
    return (1 - spatial.distance.cosine(vetor_1, vetor_2))


def ordenar_resultado(res):

    indices = []
    valores = []

    for i in range(len(res)):
        
        ind = res[i].argsort()
        val = res[i][res[i].argsort()]

        indices.append( list(reversed(ind)) )
        valores.append( list(reversed(val)) )

    return indices, valores

In [ ]:
for enu, df_pp in enumerate(lista_df_pp):
    

    # removendo a pontuação do título
    df_pp["titulo_pp"] = df_pp["titulo"].apply(lambda x: remove_pontuacao(x))

    # calculando o tamanho máximo do título
    tam_max = max(df_pp.apply(lambda row: len(row["titulo_pp"]), axis = 1))

    # calculando o BoW do título
    cv, titulo_bow = formatar_entrada_bow(df_pp['titulo_pp'], mf = tam_max)
    #np.unique(titulo_bow)



    # calculando a distância entre os vetores
    resultado = np.zeros((len(titulo_bow), len(titulo_bow)))

    for i in range(len(titulo_bow)):

        for j in range(len(titulo_bow)):

            if i == j:
                resultado[i][j] = -1
            else:
                resultado[i][j] = calcular_dis_cos(titulo_bow[i], titulo_bow[j])


    indices, valores = ordenar_resultado(resultado)

    for k in [1, 10, 50]:

        # criando uma coluna nova no df
        df_pp[f'k-{k}'] = 0
        df_pp[f'k-{k}-qtd'] = 0
        df_pp[f'k-{k}-qtd/{k}'] = 0

        for i in range(len(indices)):

            for j in range(k):
                
                if df_pp['ean'].loc[i] == df_pp['ean'].loc[indices[i][j]]:

                    df_pp[f'k-{k}'].loc[i] = 1
                    df_pp[f'k-{k}-qtd'].loc[i] += 1

            df_pp[f'k-{k}-qtd/{k}'].loc[i] = (df_pp[f'k-{k}-qtd'].loc[i])/k


    # criando uma coluna nova no df
    df_pp['match_rank'] = 0
    df_pp['1/match_rank'] = 0
    df_pp['match_rank/total'] = 0
    tam_df = df_pp.shape[0]

    # para cada linha do dataframe
    for i in range(tam_df):

        # para cada uma das distâncias encontradas rankeadas em ordem decrescente
        for cont, j in enumerate(indices[i]):
        
            # se for match
            if (df_pp['ean'].loc[i] == df_pp['ean'].loc[j]) and (i != j):

                # guarda a posição do primeiro match, lembrando que o enumerate começa em 0
                df_pp['match_rank'].loc[i] = (cont + 1)
                df_pp['1/match_rank'].loc[i] = 1/(cont + 1)
                df_pp['match_rank/total'].loc[i] = (cont + 1)/tam_df
                
                # para o for
                break

    df_pp.to_csv(f"Dados/Produtos Processados/Metricas/{arquivos[enu]}.csv", index = False)

In [ ]:
df_pp[["k-1", "k-1-qtd", "k-1-qtd/1", "k-10", "k-10-qtd", "k-10-qtd/10", "k-50", "k-50-qtd", "k-50-qtd/50", 'match_rank', 'match_rank/total', '1/match_rank']]